# Query data from the IIASA database infrastructure

The IIASA *Energy, Climate, and Environment* Program hosts a suite of **Scenario Explorer** instances and related database infrastructure to support analysis of integrated-assessment pathways in IPCC reports and model comparison projects. 
High-profile use cases include the [AR6 Scenario Explorer hosted by IIASA](https://data.ece.iiasa.ac.at/ar6) supporting the *IPCC' Sixth Assessment Report* (AR6) and the Horizon 2020 project [ENGAGE](https://data.ece.iiasa.ac.at/engage).

IIASA's [modeling platform infrastructure](http://docs.ece.iiasa.ac.at/) and the scenario apps are not only a great resource on its own, but it also allows the underlying datasets to be queried directly via a Rest API.
The **pyam** package takes advantage of this ability to allow you to easily pull data and work with it in your Python data processing and analysis workflow.

## Access and permission management for project-internal databases

By default, your can connect to all public scenario database instances.

If you have credentials to connect to a private, project-internal database instance,  
you can store this information by running the following command in a console:

```
ixmp4 login <username>

```

You will be prompted to enter your password.

<div class="alert alert-warning">

Your username and password will be saved locally in plain-text for future use!

</div>

When connecting to a database, **pyam** will automatically search for the configuration in a known location.

In [ ]:
import pyam

## Connecting to ixmp4 database instances hosted by IIASA

The *Scenario Apps* use the **ixmp4** package as a database backend.

<div class="alert alert-warning">

The *Scenario Services* team is currently migrating database instances from the legacy *Scenario Explorer* infrastructure  
to the new *Scenario Apps* services based on the **ixmp4** package.

</div>

You can list all available ixmp4 platforms hosted by IIASA using the following function.

In [ ]:
pyam.iiasa.list_platforms()

## Connecting to a (legacy) Scenario Explorer database

Accessing the database connected to a **Scenario Explorer** is done via a **Connection** object.

In [ ]:
conn = pyam.iiasa.Connection()
conn.valid_connections

## Reading data from a database hosted by IIASA

The **pyam** package can read both from **ixmp4** databases (connected to a *Scenario App*)  
and the (legacy) **Scenario Explorer** infrastructure with the same function.

In this example, we will be retrieving data from the *IAMC 1.5°C Scenario Explorer hosted by IIASA*
([link](https://data.ece.iiasa.ac.at/iamc-1.5c-explorer)),
which provides the quantitative scenario ensemble underpinning
the *IPCC Special Report on Global Warming of 1.5C* (SR15).

This can be done either via the constructor:

```
pyam.iiasa.Connection('iamc15')
```

or, if you want to query multiple databases, via the explicit `connect()` method:

```
conn = pyam.iiasa.Connection()
conn.connect('iamc15')
```

We also provide some convenience functions to shorten the amount of code you have to write. Under the hood, `read_iiasa()` is just opening a connection to a database API and sends a query to the resource.

In this tutorial, we will query specific subsets of data in a manner similar to `pyam.IamDataFrame.filter()`.

In [ ]:
df = pyam.read_iiasa(
    "iamc15",
    model="MESSAGEix*",
    variable=["Emissions|CO2", "Primary Energy|Coal"],
    region="World",
    meta=["category"],
)

Here we pulled out all times series data for model(s) that start with 'MESSAGEix' that are in the 'World' region and associated with the two named variables. We also added the meta column "category", which tells us the climate impact categorisation of each scenario as assessed in the IPCC SR15.

Let's plot CO2 emissions.

In [ ]:
ax = df.filter(variable="Emissions|CO2").plot(
    color="category", legend=dict(loc="center left", bbox_to_anchor=(1.0, 0.5))
)

And now continue doing all of your analysis!

In [ ]:
ax = df.plot.scatter(
    x="Primary Energy|Coal",
    y="Emissions|CO2",
    color="category",
    legend=dict(loc="center left", bbox_to_anchor=(1.0, 0.5)),
)

## Exploring the data resource

If you're interested in what data is available in the data source, you can use **pyam.iiasa.Connection** to do so.

In [ ]:
conn = pyam.iiasa.Connection("iamc15")

The **Connection** object has a number of useful functions for listing what's available in the data resource.
These functions follow the conventions of the **IamDataFrame** class (where possible).

A few of them are shown below.

In [ ]:
conn.models().head()

In [ ]:
conn.scenarios().head()

In [ ]:
conn.variables().head()

In [ ]:
conn.regions().head()

A number of different categorization and quantitative indicators are available for model/scenario combinations.  
These are usually called 'meta' indicators in **pyam**.

We queried the meta-indicator "category" in the above example, but there are many more.
You can get a list with the following command:

In [ ]:
conn.meta_columns.head()

You can directly query the **Connection**, which will return a **pyam.IamDataFrame**...

In [ ]:
df = conn.query(
    model="MESSAGEix*",
    variable=["Emissions|CO2", "Primary Energy|Coal"],
    region="World",
)

...so that you can directly continue with your analysis and visualization workflow using **pyam**!

In [ ]:
ax = df.filter(variable="Primary Energy|Coal").plot(
    color="scenario", legend=dict(loc="center left", bbox_to_anchor=(1.0, 0.5))
)

Loading all of a large database may take a few minutes on some connections. To save time when writing code you may reuse, you can save a local version of the database via the __lazy_read_iiasa__ function. This is given a file location as well as whatever connection options we saw above. The first time the code is run, the result is stored there, and the code will read it from there on subsequent attempts. 

In [ ]:
lazy_df = pyam.lazy_read_iiasa(
    file="./tmp/messageix_co2_coal_data.csv",
    name="iamc15",
    model="MESSAGEix*",
    variable=["Emissions|CO2", "Primary Energy|Coal"],
    region="World",
)